OpenAI GPT API Fine tuning

✨왜, 파인튜닝을 해야하는가 : 현재 gpt는 2021년 까지의 데이터만 있기에, 밀양시 리빙랩에 적용하기 위해서 필요함. 해당 답을 알려주기도 하고, 내가 원하는 format으로 답이 나오도록 하기 위해 파인튜닝 과정이 필요함

참고 티스토리
https://lsjsj92.tistory.com/m/657

파인튜닝지피티.png

https://lsjsj92.tistory.com/657

### 참고 깃허브
https://github.com/lsjsj92/recommender_system_with_Python

- OS : MacOS
- Python : Python3.9
- 라이브러리
    - OpenAI : 0.27.0
    - sentence-transformers : 2.2.2 (해당 라이브러리는 windows환경에서도 사용이 가능, Python 3.6이상과 호환이 가능)

데이터 준비(활용 데이터 셋팅) → embedding vector추출 → 코사인 유사도  계산 함수 구현 → chatGPT에 질의한 Prompt 설정 →추천 로직 구현

# 데이터 준비 : 시스템에 활용할 데이터 준비

In [ ]:
#참고 사이트에서는 정량적인 csv파일을 들고왔습니다.
movies_metadata = pd.read_csv('./~/~.csv', sep=',', dtype=str)
print(~.shape)
~.head()

In [ ]:
#참고사이트에서 정량적인 CSV파일을 들고옴
#DB에서 들고올 때 str 타입으로 들고옴
import pandas as pd
import pyodbc # DB에서 들고올 때
import csv # csv파일을 읽고 쓰고 들고올기 위해

#csv들고 올 때
들고올_데이터명 = pd.read_csv('~.csv', sep=',', dtype=str)

# DB 연결 설정
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=servername;DATABASE=databasename;UID=username;PWD=password')

# SQL 쿼리 작성(임의로 열을 적은부분으로, DB에 어떻게 저장되느냐에따라 바뀔 거 같습니다.)
sql_query = "SELECT order_id, customer_id, CONVERT(date, order_date) AS order_date FROM orders"

# 데이터 가져오기
df = pd.read_sql(sql_query, conn)

# 결과 확인
print(df.head())

#  Embedding vector 추출
- why? 텍스트의 embedding vector를 구성하여, 사용자가 날리는 질문(query)에 따라 코사인 유사도를 구하기 위해 → 가장 유사한 질문목록(답변)에 대한 정보 설명, 아이템 추천 등에 활용
- 구글에서 제공하는 developer//임베딩에 대한 설명
https://developers.google.com/machine-learning/crash-course/embeddings/video-lecture?hl=ko

## OpenAI에서 제공하는 openai.embedding활용
    - OpenAI embedding을 활용하면, 환경이 통일되어 편리 할 수 있음.

In [ ]:
openai_embedding_model = "text-embedding-ada-002"

def get_embedding(text: str, model: str) -> List[float]:
    result = openai.Embedding.create(
      model=model,
      input=text)
    return result["data"][0]["embedding"]
    
movies_metadata['openai_embeddings'] = movies_metadata['feature'].apply(lambda x : get_embedding(x, openai_embedding_model))

## HuggingFace embedding 활용
- 모델의 특징 : 가벼움, 다양한 모델로 변경 가능
- HuggingFace엣 제공하는 model들
https://huggingface.co/models

In [ ]:
model = SentenceTransformer("사용하고 싶은 모델")
#사이트를 보니 SnetenceTransformer
movies_metadata['hf_embeddings'] = movies_metadata['feature'].apply(lambda x : model.encode(x))

# 코사인 유사도 구현
- huggingface의 sentence_transformer에 있는 utils.pytorch_cos_sim을 활용
- 질문(query) → 정보 획득 → model.encode에 데이터 업로드 → 질문(query)가 가지고 있는 embddong 정보 가져오기 → torch.topk를 활용하여 top 개수를 가져옴

In [ ]:
import torch
from sentence_transformers import util

def get_query_sim_top_k(query, model, df, top_k):
# 입력된 쿼리를 Sentence Transformers 모델을 사용하여 인코딩
    query_encode = model.encode(query)
# 데이터프레임(df)에서 문서의 임베딩 벡터를 가져와 쿼리 벡터와의 코사인 유사도를 계산
# 이를 위해, PyTorch의 cosine similarity 함수를 사용
    cos_scores = util.pytorch_cos_sim(query_encode, df['hf_embeddings'])[0]
# 상위 K개의 유사도 점수를 반환합니다.
    top_results = torch.topk(cos_scores, k=top_k)
    return top_results
# top_results를 반환

# ChatGPT를 활용한 추천 시스템에 활용한 Prompt 환경 구축

- engineering process image
    1. 추천을 원하는 것인가
    2. 추가 정보를 더 알고 싶은 것인가
    3. 상세한 설명을 원하는 것인가

##  **prompt를 [추천을 원하는 것인가][추가 정보를 더 알고 싶은가][상세한 설명을 원하는 것인가]로 나누는 이유**
    - 사용자가 어떤 query를 입력함
    - chatgpt는 해당 query에 따라 사용자의 의도를 파악
    - 또한, ChatGPT는 사용자 의도에 따라 자연스러운 도움 메세지 생성
    - huggingface의 embedding을 기반으로 유사도를 계산해 어떤 콘텐츠를 추천해줄 때 도움 메세지 뒤에 콘텐츠 추천 정보 제공
    - 이러한 사용자 이력을 기반으로 상세 정보 설명등에 활용
    - chatgpt를 활용한 추천 시스템 주제를 설정하기 위한 engineering 준비

In [ ]:
#prompt를 작성하기 위한 코드
def print_msg(msg):
    completion = openai.ChatCompletion.create(
                    model="gpt-3.5-turbo",
                    messages=msg
                    )
    return completion['choices'][0]['message']['content']